# KLUE Relation Extraction - 학습 전 베이스라인 평가

이 노트북은 **학습 없이** 얻을 수 있는 기준 성능을 확인합니다.
- 다수 클래스(majority) 예측
- 랜덤 예측 (균등/분포 기반)

결과는 파인튜닝 이후 성능과 비교하는 기준점으로 사용합니다.

## 1. 환경 설정

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# 재현성
SEED = 42
np.random.seed(SEED)
print(f"✅ Seed fixed: {SEED}")

✅ Seed fixed: 42


## 2. 결과 저장 디렉터리 설정

In [2]:
NOTEBOOK_NAME = "05_klue_re_no_train_baseline"
RESULT_DIR = Path("temp_result") / NOTEBOOK_NAME
RESULT_DIR.mkdir(parents=True, exist_ok=True)


def _to_builtin(obj):
    if isinstance(obj, dict):
        converted = {}
        for k, v in obj.items():
            kk = k
            try:
                kk = kk.item()
            except Exception:
                pass
            if isinstance(kk, (dict, list, tuple)):
                kk = str(kk)
            converted[kk] = _to_builtin(v)
        return converted
    if isinstance(obj, (list, tuple)):
        return [_to_builtin(v) for v in obj]
    try:
        return obj.item()
    except Exception:
        return obj


def save_json(filename, obj):
    path = RESULT_DIR / filename
    path.write_text(json.dumps(_to_builtin(obj), ensure_ascii=False, indent=2), encoding="utf-8")
    return path

print(f"✅ 결과 저장 경로: {RESULT_DIR}")

✅ 결과 저장 경로: temp_result/05_klue_re_no_train_baseline


## 3. 데이터 로드

In [3]:
train_df = pd.read_csv('data/klue_re_train.csv')
valid_df = pd.read_csv('data/klue_re_validation.csv')

print(f"Train: {len(train_df):,} / Valid: {len(valid_df):,}")
print("컬럼:", train_df.columns.tolist())

Train: 32,470 / Valid: 7,765
컬럼: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source']


## 4. 평가 지표 정의
- Accuracy
- Macro F1
- Micro F1

In [4]:
from sklearn.metrics import accuracy_score, f1_score


def eval_metrics(y_true, y_pred):
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "macro_f1": float(f1_score(y_true, y_pred, average='macro')),
        "micro_f1": float(f1_score(y_true, y_pred, average='micro')),
    }

## 5. 베이스라인 1: 다수 클래스(majority) 예측

In [5]:
major_label = train_df['label'].value_counts().idxmax()

valid_pred_major = np.full(len(valid_df), major_label)
metrics_major = eval_metrics(valid_df['label'], valid_pred_major)

print("Majority label:", major_label)
print(metrics_major)

save_json("baseline_majority.json", {
    "major_label": int(major_label),
    "metrics": metrics_major
})

Majority label: 0
{'accuracy': 0.5963940759819704, 'macro_f1': 0.024905883618371517, 'micro_f1': 0.5963940759819704}


PosixPath('temp_result/05_klue_re_no_train_baseline/baseline_majority.json')

## 6. 베이스라인 2: 랜덤 예측 (균등 분포)

In [6]:
labels = sorted(train_df['label'].unique())
valid_pred_uniform = np.random.choice(labels, size=len(valid_df), replace=True)
metrics_uniform = eval_metrics(valid_df['label'], valid_pred_uniform)

print(metrics_uniform)

save_json("baseline_random_uniform.json", {
    "metrics": metrics_uniform
})

{'accuracy': 0.03193818415969092, 'macro_f1': 0.016523317185787874, 'micro_f1': 0.03193818415969092}


PosixPath('temp_result/05_klue_re_no_train_baseline/baseline_random_uniform.json')

## 7. 베이스라인 3: 랜덤 예측 (학습 분포 기반)

In [7]:
label_counts = train_df['label'].value_counts().sort_index()
probs = label_counts.values / label_counts.values.sum()

valid_pred_weighted = np.random.choice(label_counts.index, size=len(valid_df), replace=True, p=probs)
metrics_weighted = eval_metrics(valid_df['label'], valid_pred_weighted)

print(metrics_weighted)

save_json("baseline_random_weighted.json", {
    "metrics": metrics_weighted
})

{'accuracy': 0.19523502897617515, 'macro_f1': 0.028611222209651697, 'micro_f1': 0.19523502897617515}


PosixPath('temp_result/05_klue_re_no_train_baseline/baseline_random_weighted.json')

## 8. 요약
- 학습 없이 얻은 기준 성능을 기록함
- 이후 파인튜닝 성능과 비교하여 학습 효과를 검증할 수 있음

- **다수 클래스(majority) 예측**은 accuracy/micro F1 **0.596**으로 가장 높지만, **macro F1 0.0249**로 거의 학습이 없는 수준임. 이는 **no_relation 비중이 큰 클래스 불균형** 때문에 생기는 착시임.
- **균등 랜덤 예측**은 accuracy/micro F1 **0.0319**, macro F1 **0.0165**로 사실상 의미 없는 기준선.
- **분포 기반 랜덤 예측**은 accuracy/micro F1 **0.195**, macro F1 **0.0286**으로 약간 개선되나, 여전히 모든 관계를 균형 있게 맞추지 못함.
- 결론적으로 KLUE-RE는 **macro F1 중심 평가가 필수**이며, 이후 학습/개선 단계에서는 **소수 클래스 성능 향상**이 핵심 과제가 됨.